# Practical example. Audiobooks #

Create a machine learning algorithm that can predict if a customer will buy again. Classification problem with 2 classes. Wont buy and will buy.
Use the Audiobooks_data.csv dataset. 

Action Plan:
1) Preprocess the data.
    - Balance the dataset
    - Divide the dataset in training, validation, and test. (Prevent overfitting)
    - Save the data in a tensor friendly format. (.npz file type)
2) Create the machine learning algorithm


We will need to preprocess a bit. This is the relevant code, but refers to data engineering more than machine learning.

## Extract the data from csv ##

In [1]:
import numpy as np
from sklearn import preprocessing # pip install scikit-learn

raw_csv_data = np.loadtxt('Resources/Audiobooks_data.csv', delimiter=',')
unscaled_inputs_all = raw_csv_data[:,1:-1]
targets_all = raw_csv_data[:,-1]

## Balance the dataset ##

In [2]:
# Count the number of targets that are '1'. We picked 1 instead of 0 because we know there are less 1s than 0s. Keep as many 0s as 1s
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []

for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter+=1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)
            
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

## Standardize the inputs ##

In [3]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

## Shuffle the data ##

In [4]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

## Split the dataset into train, validation, and test

In [5]:
samples_count = shuffled_inputs.shape[0]

# Use the 80-10-10 split for train, validation, and test
train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1808.0 3579 0.5051690416317407
222.0 447 0.4966442953020134
207.0 448 0.46205357142857145


## Save the tree datasets in *.npz ##

In [6]:
np.savez('Resources/Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Resources/Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Resources/Audiobooks_data_test', inputs=test_inputs, targets=test_targets)

# -------------------------------- Create the Machine Learning Algorithm --------------------------------

In [7]:
import tensorflow as tf

## Data

In [18]:
npz = np.load('Resources/Audiobooks_data_train.npz')
train_inputs = npz['inputs'].astype(np.float64)
train_targets = npz['targets'].astype(np.int64)
#print(type(npz))
#print(f"Input type: {str(type(train_inputs))}    target type: {str(type(train_targets))}")

npz = np.load('Resources/Audiobooks_data_validation.npz')
validation_inputs = npz['inputs'].astype(np.float64)
validation_targets = npz['targets'].astype(np.int64)

# test data hw
npz = np.load('Resources/Audiobooks_data_test.npz')
test_inputs = npz['inputs'].astype(np.float64)
test_targets = npz['targets'].astype(np.int64)

## Model

Outline, optimizers, loss, early stopping and training

In [29]:
# Outline
input_size = 10
output_size = 2
hidden_layer_size = 50

model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

# Optimizer and loss function
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# early stopping and training
batch_size = 100
max_epochs = 100
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(
    train_inputs, 
    train_targets, 
    batch_size=batch_size, 
    epochs=max_epochs,
    callbacks=[early_stopping],
    validation_data=(validation_inputs, validation_targets),
    verbose=2
)

Epoch 1/100
36/36 - 0s - 13ms/step - accuracy: 0.6695 - loss: 0.5967 - val_accuracy: 0.7338 - val_loss: 0.5014
Epoch 2/100
36/36 - 0s - 2ms/step - accuracy: 0.7617 - loss: 0.4631 - val_accuracy: 0.7584 - val_loss: 0.4368
Epoch 3/100
36/36 - 0s - 2ms/step - accuracy: 0.7868 - loss: 0.4123 - val_accuracy: 0.7606 - val_loss: 0.4092
Epoch 4/100
36/36 - 0s - 2ms/step - accuracy: 0.7826 - loss: 0.3930 - val_accuracy: 0.7830 - val_loss: 0.3843
Epoch 5/100
36/36 - 0s - 2ms/step - accuracy: 0.8036 - loss: 0.3753 - val_accuracy: 0.7919 - val_loss: 0.3772
Epoch 6/100
36/36 - 0s - 2ms/step - accuracy: 0.8128 - loss: 0.3640 - val_accuracy: 0.7942 - val_loss: 0.3677
Epoch 7/100
36/36 - 0s - 2ms/step - accuracy: 0.8164 - loss: 0.3561 - val_accuracy: 0.7964 - val_loss: 0.3622
Epoch 8/100
36/36 - 0s - 2ms/step - accuracy: 0.8111 - loss: 0.3525 - val_accuracy: 0.7942 - val_loss: 0.3543
Epoch 9/100
36/36 - 0s - 2ms/step - accuracy: 0.8122 - loss: 0.3490 - val_accuracy: 0.8121 - val_loss: 0.3495
Epoch 10/

## Test the model

In [31]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7719 - loss: 0.3743 


In [37]:
print(f"\nTest loss: {test_loss:.2f}. Test accuracy: {test_accuracy*100:.2f}%")


Test loss: 0.37. Test accuracy: 79.02%
